In [ ]:
!pip install tensorflow-decision-forests

In [33]:
import tensorflow_decision_forests as tfdf
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report, f1_score
import random

In [34]:
def train_model(data):
  train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(data, label="subcategory")
  model = tfdf.keras.RandomForestModel()
  start = datetime.now()
  model.fit(train_ds)
  end = datetime.now()
  print(end-start)
  return model

In [35]:
def data_conversion(predictions_ds):
  predictions_df=pd.DataFrame(predictions_ds)
  predictions_df['subcategory']=predictions_df.idxmax(1)
  col='subcategory'
  predictions_df=predictions_df.loc[:, predictions_df.columns == col]
  return predictions_df

In [36]:
def fix_data(data):
  try:
    data.pop('Unnamed: 0')
    data.pop('Unnamed: 0.1')
  except:
    pass
  return data

In [37]:
def remove_entries(df,value):
  indexes = df.index[df['subcategory'] == value].tolist()
  indexes=indexes[:len(indexes)-2]
  df=df.drop(index=indexes)
  return df

In [38]:
def remove_entries2(df,value):
  df=df.loc[(df['subcategory'] != value)]
  return df

In [39]:
def majority_vote(device1_predictions,device2_predictions,device3_predictions,device4_predictions):
  start=datetime.now()
  combined=mode_series = pd.concat([device1_predictions.rename(columns={"subcategory": "s1"}), device2_predictions.rename(columns={"subcategory": "s2"}), device3_predictions.rename(columns={"subcategory": "s3"}), device4_predictions.rename(columns={"subcategory": "s4"})], axis=1)
  modes = combined[['s1', 's2', 's3', 's4']].mode(axis=1, numeric_only=True, dropna=True)[0]
  modes=modes.astype(int)
  end=datetime.now()
  print('Global train time', end-start)
  return modes

In [40]:
device1_data=fix_data(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Second semester/Data/label_split/device1.csv'))
device2_data=fix_data(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Second semester/Data/label_split/device2.csv'))
device3_data=fix_data(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Second semester/Data/label_split/device3.csv'))
device4_data=fix_data(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Second semester/Data/label_split/device4.csv'))

In [ ]:
#Zero day attacks
device1_data=remove_entries(device1_data,0)
device1_data=remove_entries(device1_data,1)

device2_data=remove_entries(device2_data,2)
device2_data=remove_entries(device2_data,3)

device3_data=remove_entries(device3_data,4)
device3_data=remove_entries(device3_data,5)

device4_data=remove_entries(device4_data,6)
device4_data=remove_entries(device4_data,7)
device4_data=remove_entries(device4_data,8)

In [ ]:
#Zero day attacks
device1_data=remove_entries2(device1_data,0)
device1_data=remove_entries2(device1_data,1)

device2_data=remove_entries2(device2_data,2)
device2_data=remove_entries2(device2_data,3)

device3_data=remove_entries2(device3_data,4)
device3_data=remove_entries2(device3_data,5)

device4_data=remove_entries2(device4_data,6)
device4_data=remove_entries2(device4_data,7)
device4_data=remove_entries2(device4_data,8)

In [41]:
test_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Second semester/Data/random_split/test.csv')
test_ds= tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label="subcategory")
test_y=test_data.pop('subcategory')

In [42]:
device1=train_model(device1_data)
device2=train_model(device2_data)
device3=train_model(device3_data)
device4=train_model(device4_data)

Use /tmp/tmpmluestm2 as temporary training directory
Reading training dataset...


Training dataset read in 0:00:00.681226. Found 1156 examples.
Training model...
Model trained in 0:00:00.177859
Compiling model...
Model compiled.


0:00:01.292568
Use /tmp/tmpb75wm36o as temporary training directory
Reading training dataset...


Training dataset read in 0:00:54.619638. Found 1540025 examples.
Training model...
Model trained in 0:05:53.839761
Compiling model...
Model compiled.


0:07:23.357071
Use /tmp/tmpw4e_335l as temporary training directory
Reading training dataset...
Training dataset read in 0:00:29.144437. Found 827657 examples.
Training model...
Model trained in 0:01:52.940327
Compiling model...
Model compiled.
0:02:23.322747
Use /tmp/tmpztpvqst5 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:22.886326. Found 558674 examples.
Training model...
Model trained in 0:02:02.720542
Compiling model...


Model compiled.
0:03:23.000494


In [43]:
device1_predictions=data_conversion(device1.predict(test_ds))
device2_predictions=data_conversion(device2.predict(test_ds))
device3_predictions=data_conversion(device3.predict(test_ds))
device4_predictions=data_conversion(device4.predict(test_ds)) 

732/732 [==============================] - 37s 50ms/step


In [44]:
modes=majority_vote(device1_predictions, device2_predictions, device3_predictions, device4_predictions)

Global train time 0:02:50.165091


In [45]:
sk_report = classification_report(digits=4,y_true=test_y,y_pred=modes)
sk_report

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0     0.0001    1.0000    0.0002        87\n           1     0.0000    0.0000    0.0000       225\n           2     0.0000    0.0000    0.0000    189683\n           3     0.0000    0.0000    0.0000    195579\n           4     0.0000    0.0000    0.0000       285\n           5     0.0000    0.0000    0.0000    206616\n           6     0.0000    0.0000    0.0000    122982\n           7     0.0000    0.0000    0.0000      3576\n           8     0.0000    0.0000    0.0000     12846\n\n    accuracy                         0.0001    731879\n   macro avg     0.0000    0.1111    0.0000    731879\nweighted avg     0.0000    0.0001    0.0000    731879\n'

In [46]:
sk_report = classification_report(digits=4,y_true=test_y,y_pred=device1_predictions)
sk_report

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0     0.0001    1.0000    0.0002        87\n           1     0.0000    0.0000    0.0000       225\n           2     0.0000    0.0000    0.0000    189683\n           3     0.0000    0.0000    0.0000    195579\n           4     0.0000    0.0000    0.0000       285\n           5     0.0000    0.0000    0.0000    206616\n           6     0.0000    0.0000    0.0000    122982\n           7     0.0000    0.0000    0.0000      3576\n           8     0.0000    0.0000    0.0000     12846\n\n    accuracy                         0.0001    731879\n   macro avg     0.0000    0.1111    0.0000    731879\nweighted avg     0.0000    0.0001    0.0000    731879\n'

In [47]:
sk_report = classification_report(digits=4,y_true=test_y,y_pred=device2_predictions)
sk_report

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0     0.0000    0.0000    0.0000        87\n           1     0.0000    0.0000    0.0000       225\n           2     0.9988    1.0000    0.9994    189683\n           3     0.3609    1.0000    0.5303    195579\n           4     0.0000    0.0000    0.0000       285\n           5     0.0000    0.0000    0.0000    206616\n           6     0.0000    0.0000    0.0000    122982\n           7     0.0000    0.0000    0.0000      3576\n           8     0.0000    0.0000    0.0000     12846\n\n    accuracy                         0.5264    731879\n   macro avg     0.1511    0.2222    0.1700    731879\nweighted avg     0.3553    0.5264    0.4007    731879\n'

In [48]:
sk_report = classification_report(digits=4,y_true=test_y,y_pred=device3_predictions)
sk_report

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0     0.0000    0.0000    0.0000        87\n           1     0.0000    0.0000    0.0000       225\n           2     0.0000    0.0000    0.0000    189683\n           3     0.0000    0.0000    0.0000    195579\n           4     0.0007    1.0000    0.0015       285\n           5     0.5970    1.0000    0.7476    206616\n           6     0.0000    0.0000    0.0000    122982\n           7     0.0000    0.0000    0.0000      3576\n           8     0.0000    0.0000    0.0000     12846\n\n    accuracy                         0.2827    731879\n   macro avg     0.0664    0.2222    0.0832    731879\nweighted avg     0.1685    0.2827    0.2111    731879\n'

In [49]:
sk_report = classification_report(digits=4,y_true=test_y,y_pred=device4_predictions)
sk_report

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0     0.0000    0.0000    0.0000        87\n           1     0.0000    0.0000    0.0000       225\n           2     0.0000    0.0000    0.0000    189683\n           3     0.0000    0.0000    0.0000    195579\n           4     0.0000    0.0000    0.0000       285\n           5     0.0000    0.0000    0.0000    206616\n           6     0.1719    1.0000    0.2934    122982\n           7     1.0000    1.0000    1.0000      3576\n           8     0.9933    1.0000    0.9966     12846\n\n    accuracy                         0.1905    731879\n   macro avg     0.2406    0.3333    0.2544    731879\nweighted avg     0.0512    0.1905    0.0717    731879\n'